In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
from math import tau
from scipy.io import wavfile

pylab.rcParams['figure.figsize'] = 12, 5
𝜏 = tau

In [ ]:
hz = 44100
T = 5             # number of seconds
N = hz * T  # number of samples
n = arange(N)     # samples vector
t = n / hz        # time vector, in seconds
start_hz = 10
stop_hz = 5000

In [ ]:
if 1:  # linear chirp
    chirp_rate = (stop_hz - start_hz) / T
    a = np.sin(𝜏 * (chirp_rate * t * t / 2.0 + start_hz * t))
else:  # exponential chirp
    chirp_rate = (stop_hz / start_hz) ** (1 / T)
    a = np.sin(𝜏 * start_hz * (chirp_rate ** t - 1) / np.log(chirp_rate))
plot(t, a, ',')

In [ ]:
from numpy.fft import fft
f = fft(a)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(n/T, abs(f), ',')
ax2.plot(n/T, np.angle(f), ',')
[ax.set_xlim(10, hz/2 - 10) for ax in (ax1, ax2)];
[ax.set_xscale('log') for ax in (ax1, ax2)];

In [ ]:
print(len(a) * 1.0 / hz, 'seconds of audio')
left = 8000 * a
right = 0 * a
stereo = np.array([left, right]).astype(np.int16).T
wavfile.write('sweep.wav', hz, stereo)

# Recording

Notes:

```
arecord -f cd -c 1 audio-in.wav -D "HDA Intel PCH"

--duration 10
```

In [ ]:
#output_device = 'plughw:CARD=PCH,DEV=0'
output_device = 'plughw:CARD=USB,DEV=0'
input_device = 'plughw:CARD=USB,DEV=0'

In [ ]:
! aplay -D {output_device} sweep.wav

In [ ]:
! (sleep 1; aplay -D {output_device} sweep.wav) & arecord -f cd -c 2 -D {input_device} --duration={T+2} audio-in.wav
! ls -l audio-in.wav

In [ ]:
hz, channels = wavfile.read('audio-in.wav')
left, right = channels.T
signal = right

In [ ]:
plot(signal)

In [ ]:
# Overview plot, to see everything that is going on.
N = len(signal)
f = fft(signal)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(hz * arange(N) / N, abs(f), ',')
ax2.plot(hz * arange(N) / N, np.angle(f), ',')
[ax.set_xlim(10, hz/2 - 10) for ax in (ax1, ax2)];

In [ ]:
# Plot narrowed to where we expect signal.
N = len(signal)
f = fft(signal)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(hz * arange(N) / N, abs(f), ',')
ax2.plot(hz * arange(N) / N, np.angle(f), ',')
[ax.set_xlim(start_hz, stop_hz) for ax in (ax1, ax2)];
[ax.set_xscale('log') for ax in (ax1, ax2)];

In [ ]:
# Compare!

hz_i, channels = wavfile.read('sweep.wav')
left, right = channels.T
input_signal = left

hz_o, channels = wavfile.read('audio-in.wav')
left, right = channels.T
output_signal = right

assert hz_i == hz_o

Ni = len(input_signal)
fi = abs(fft(input_signal))

No = len(output_signal)
fo = abs(fft(output_signal))

frequency = hz_o * arange(No) / No
print(frequency.shape)
print(fo.shape)
print(arange(Ni).shape)
fi = np.interp(frequency, hz_i * arange(Ni) / Ni, abs(fi))
print(fi.shape)

mask = (frequency >= start_hz) & (frequency <= stop_hz)
frequency = frequency[mask]
fi = fi[mask]
fo = fo[mask]

ratio = fo / fi

fig, [ax1, ax2] = plt.subplots(2, 1)
ax1.plot(frequency, fi, ',')
ax1.plot(frequency, fo, ',')
ax2.plot(frequency, ratio, ',')
# numpy.interp(x, xp, fp, left=None, right=None, period=None)
#[ax.set_xlim(start_hz * 2, stop_hz * 0.6) for ax in [ax]];
[ax.set_xscale('log') for ax in [ax1, ax2]];

In [ ]:
# def moving_average(a, n=3) :
#     ret = np.cumsum(a, dtype=float)
#     ret[n:] = ret[n:] - ret[:-n]
#     return ret[n - 1:] / n

In [ ]:
# from numpy.fft import fft
# f = fft(a)
# fig, (ax1, ax2) = plt.subplots(2, 1)
# ax1.plot(moving_average(abs(f), 10), ',')
# a = np.angle(f)
# ax2.plot(a, ',')

In [ ]:
# from numpy.fft import fft
# f = fft(a)
# #print(f.imag)
# fig, (ax1, ax2) = plt.subplots(2, 1)
# ax1.plot(moving_average(abs(f), 10), ',')
# ax1.set_xlim(10, 22050)
# #ax1.plot(f.imag)
# a = np.angle(f)
# ax2.plot(a, ',')
# ax2.set_xlim(10, 22050)